In [1]:
from pyspark import SparkConf, SparkContext

conf = SparkConf().setMaster('local').setAppName('partition')
spark = SparkContext(conf = conf).getOrCreate()
spark

24/12/05 10:38:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


<SparkContext master=local appName=partition>

### 사용자가 지정하는 파티션
-partitionBy()

In [2]:
pairs = spark.parallelize(
                            [
                                1,2,3,4,2,4,1
                            ]   
).map(lambda x : (x,x))

In [4]:
pairs.take(3)

[(1, 1), (2, 2), (3, 3)]

In [6]:
#어느 파티션에 있는 지 확인(glom())
pairs.glom().collect()

[[(1, 1), (2, 2), (3, 3), (4, 4), (2, 2), (4, 4), (1, 1)]]

In [8]:
pairs.partitionBy(2).glom().collect()

[[(2, 2), (4, 4), (2, 2), (4, 4)], [(1, 1), (3, 3), (1, 1)]]

In [9]:
pairs.partitionBy(3).glom().collect()

[[(3, 3)], [(1, 1), (4, 4), (4, 4), (1, 1)], [(2, 2), (2, 2)]]

In [10]:
pairs.partitionBy(4).glom().collect()

[[(4, 4), (4, 4)], [(1, 1), (1, 1)], [(2, 2), (2, 2)], [(3, 3)]]

In [17]:
# 파티션별로 규칙에 따라 데이터를 넣을 수 있다. > lambda 식으로 정의
# 파티션 배치함수
pairs.partitionBy(2, lambda x : x%2).glom().collect()

[[(2, 2), (4, 4), (2, 2), (4, 4)], [(1, 1), (3, 3), (1, 1)]]

In [20]:
# 규칙을 가진 데이터를 persist해놓으면 reduce할때 성능이 좋음
new_pairs = pairs.partitionBy(2, lambda x : x%2).persist()
new_pairs.collect()

[(2, 2), (4, 4), (2, 2), (4, 4), (1, 1), (3, 3), (1, 1)]

In [21]:
spark.stop()